## Context
This notebook does some basic analysis on existing data from MyAnimeList 

In [2]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from collections import defaultdict
import numpy as np
import scipy
from scipy.sparse.linalg import svds
import surprise as sp
import time
print("Setup Complete")

Setup Complete


This is mostly to do some exploratory data analysis (EDA) in the anime dataset. It's to help me understand the type of data I am dealing with so I can start working on an anime recommender.
I hope to have a good idea on what should be the correct label.

In [4]:
UsersDF = pd.read_csv('./data/full/users_cleaned.csv')
AnimesDF = pd.read_csv('./data/full/anime_cleaned.csv')
ScoresDF = pd.read_csv('./data/full/animelists_cleaned.csv')

KeyboardInterrupt: 

I should be able to use score<sub>review</sub> to be label. But bear in mind that the score will only appear for animes which the person has watched.

profiles <sub>favourites_anime</sub> could work too.